# Step 2 ES Feature Individual Level
## Reviws and Rating Extraction 

Right now we recover a historic snapshot of ratings.   
In the future we can add NLP pieces(such as count of certain words). 

Input:
- close rate, monthly:  installer_close_rates.csv 
- reviews, by piece: installer_review_data_20180410.csv 
- reviews sentiment score, per each review 

Output: 
- 'oct29_closerate_withratingcounts.csv' 

Uplate: Jan 17, 2020




## Get the Input data 

In [2]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import os
#laptop 
#os.chdir('C:/Users/herbie huang/Google Drive/Current_SolarResearch/working_2020')
#office 
#os.chdir('C:/Users/herbieh/Google Drive/Current_SolarResearch/working')
#%%
close=pd.read_csv('C:/Users/herbie huang/Google Drive/Current_SolarResearch/RawData_EnergySage/Lock_ES_RawData/installer_close_rates.csv',sep=',')

#%%
reviews=pd.read_csv('C:/Users/herbie huang/Google Drive/Current_SolarResearch/RawData_EnergySage/Lock_ES_RawData/installer_review_data_20180410.csv',sep=',',escapechar='\\')


Close Rate (ES Installers)

In [2]:
#%%
#close rate have many NAs, and year_month are not integer 

close = close[np.isfinite(close['Year_month'])]
close['Year_month']=pd.to_numeric(arg=close['Year_month'],downcast='integer')
#%%
close['year_month']=close['Year_month']
#%% relace na with 0 
close[['quotes_count','winning_quotes_count']]=close[['quotes_count','winning_quotes_count']].fillna(0)
#%%
close['year_month']=pd.to_datetime(close['Year_month'],format='%Y%m',errors='coerce')
close['closeraterecord_id']=close.index
#now the year _month colun are set to the correct time format 

In [3]:
close[50:60]

,Unnamed: 0,installer_id,Year_month,quotes_count,winning_quotes_count,year_month,closeraterecord_id
50,50,424,201710,238.0,4.0,2017-10-01,50
51,51,424,201711,211.0,3.0,2017-11-01,51
52,52,424,201712,134.0,4.0,2017-12-01,52
53,53,424,201801,227.0,0.0,2018-01-01,53
54,54,424,201802,245.0,0.0,2018-02-01,54
55,55,424,201803,306.0,3.0,2018-03-01,55
56,56,424,201804,78.0,0.0,2018-04-01,56
57,57,551,201312,1.0,1.0,2013-12-01,57
58,58,551,201403,2.0,0.0,2014-03-01,58
59,59,551,201404,1.0,0.0,2014-04-01,59


Note this still has gaps in time. When we append the reviews score it will also have gap. 

### Reviews data merge with sentiment scores 

In [1]:
reviews.head()

NameError: name 'reviews' is not defined

In [5]:
reviews_sent_scores=pd.read_stata('reviews_sent_score_jan16_2020.dta')

In [6]:
reviews_sent_scores.head()

,id,installer_id,overall_rating,sent_score
0,1,494,1,0.4939
1,2,556,5,0.7351
2,3,857,5,0.6893
3,4,20292,5,0.7783
4,5,857,5,0.8720


In [7]:
reviews=pd.merge(left=reviews,right=reviews_sent_scores,on=['id','installer_id','overall_rating'],how='inner')

In [8]:
reviews.tail()

,id,installer_id,subject,body,active,authenticated,date_created,overall_rating,number_quotes_submitted,number_winning_quotes,sent_score
3382,3392,3247,AVOID this company,"Terrible service, difficult to deal with, wait...",1,1,2018-04-09 15:34:12,1,0,0,-0.6808
3383,3393,21242,Altair is amazing! Look no further.,This was our second solar installation. After ...,1,1,2018-04-09 16:05:47,5,842,14,0.9704
3384,3394,21269,Installation of Solar Pqnels,In April 2017 Solar Five Company installed 32 ...,1,1,2018-04-09 19:49:58,5,276,3,0.6597
3385,3395,21711,"Professional, cost-effective and easy for me",Pinnacle installed our solar panels and I coul...,1,0,2018-04-09 21:57:33,5,10,0,0.6588
3386,3396,21866,Quality Install,I signed up with America Green back in Februar...,1,1,2018-04-10 19:04:47,4,0,0,0.8689


## Work on create the data

In [10]:
close['closeraterecord_id'][:6]

0    0
1    1
2    2
3    3
4    4
5    5
Name: closeraterecord_id, dtype: int64

In [16]:
reviews.tail().sent_score.mean()

0.49540001153945923

In [21]:
#%%
#%%
reviews['date_created']=pd.to_datetime(reviews['date_created'])
#%%
allratings=[]
collect=close
avgrating=[]
count=[]
data_exp=[]
#%%
def countX(lst, x):
    count = 0
    for ele in lst:
        if (ele == x):
            count = count + 1
    return count
for i in close['closeraterecord_id']:
    temp_seedate=close['year_month'].loc[i]
    temp_installer=close['installer_id'].loc[i]
    temp_reviews_step1=reviews[reviews['installer_id']==temp_installer]
    # step 1 is to slice the reviews that only relevant to this installer as temp_installer is the id 
    temp_reviews_step2=temp_reviews_step1[reviews['date_created']<=temp_seedate]
    #step 2 is to slice the reviews that happened before the "see date" (temp_seedate)
    temp_ratings=temp_reviews_step2['overall_rating']
    print(temp_ratings)
    temp_sent_scores=temp_reviews_step2.sent_score
    temp_count5=countX(temp_ratings,5)
    temp_count4=countX(temp_ratings,4) 
    temp_count3=countX(temp_ratings,3)
    temp_count2=countX(temp_ratings,2)
    temp_count1=countX(temp_ratings,1)
    temp_avg_sent_score=temp_sent_scores.mean()
   # temp_countstars=[temp_count5,temp_count4,temp_count3,temp_count2,temp_count1]
   # count.append(temp_countstars)
    temp_dataexp=[i,temp_count5,temp_count4,temp_count3,temp_count2,temp_count1,temp_avg_sent_score]
    data_exp.append(temp_dataexp)


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Series([], Name: overall_rating, dtype: int64)
Series([], Name: overall_rating, dtype: int64)
Series([], Name: overall_rating, dtype: int64)
Series([], Name: overall_rating, dtype: int64)
Series([], Name: overall_rating, dtype: int64)
493    5
Name: overall_rating, dtype: int64
493    5
Name: overall_rating, dtype: int64
493    5
731    5
Name: overall_rating, dtype: int64
493    5
731    5
Name: overall_rating, dtype: int64
493    5
731    5
Name: overall_rating, dtype: int64
493     5
731     5
1037    1
Name: overall_rating, dtype: int64
493     5
731     5
1037    1
Name: overall_rating, dtype: int64
493     5
731     5
1037    1
1147    1
Name: overall_rating, dtype: int64
493     5
731     5
1037    1
1147    1
Name: overall_rating, dtype: int64
493     5
731     5
1037    1
1147    1
Name: overall_rating, dtype: int64
493     5
731     5
1037    1
1147    1
Name: overall_rating, dtype: int64
493     5
731     5
1037    1
1147    1
Name: overall_rating, dtype: int64
Series([], Na

In [25]:
data_exp[-5:-1]

[[6406, 6, 0, 0, 0, 0, 0.5665500164031982],
 [6407, 0, 0, 0, 0, 0, nan],
 [6408, 2, 0, 0, 0, 0, 0.6549999713897705],
 [6409, 0, 0, 0, 0, 0, nan]]

In [26]:
data_exp_df=pd.DataFrame(data=data_exp,columns=['i', 'five','four','three','two','one','avg_sent_score'])
#%%

In [28]:
data_exp_df.tail()

,i,five,four,three,two,one,avg_sent_score
6406,6406,6,0,0,0,0,0.56655
6407,6407,0,0,0,0,0,NaN
6408,6408,2,0,0,0,0,0.65500
6409,6409,0,0,0,0,0,NaN
6410,6410,3,0,0,0,0,0.69600


In [30]:
close.tail()

,Unnamed: 0,installer_id,Year_month,quotes_count,winning_quotes_count,year_month,closeraterecord_id
6406,6406,22992,201804,10.0,0.0,2018-04-01,6406
6407,6407,22997,201803,28.0,0.0,2018-03-01,6407
6408,6408,22997,201804,4.0,0.0,2018-04-01,6408
6409,6409,23027,201803,2.0,0.0,2018-03-01,6409
6410,6410,23027,201804,15.0,0.0,2018-04-01,6410


In [31]:
merged_new_closerecord=pd.merge(left=data_exp_df,right=close,left_on='i',right_on='closeraterecord_id',how='inner')

In [32]:
merged_new_closerecord.head()

,i,five,four,three,two,one,avg_sent_score,Unnamed: 0,installer_id,Year_month,quotes_count,winning_quotes_count,year_month,closeraterecord_id
0,0,0,0,0,0,0,NaN,0,108,201506,27.0,0.0,2015-06-01,0
1,1,0,0,0,0,0,NaN,1,108,201601,12.0,0.0,2016-01-01,1
2,2,0,0,0,0,0,NaN,2,108,201602,3.0,0.0,2016-02-01,2
3,3,0,0,0,0,0,NaN,3,363,201603,20.0,0.0,2016-03-01,3
4,4,0,0,0,0,0,NaN,4,363,201604,8.0,0.0,2016-04-01,4


In [33]:
pwd

'C:\\Users\\herbie huang\\Google Drive\\Current_SolarResearch\\working_2020'

In [34]:
merged_new_closerecord.to_csv('closerate_withratingcounts_sent_score_jan2020.csv')

## manually change the closerate_withratingcounts_sent_score_jan2020.csv name to closerate_withratingcounts_sent_score.csv for __stata__ 

## Use Stata for Fill Time Gaps 
1. Convert them to non-gapped data use stata "es_monthly.do" 

In [15]:
merged_new_closerecord.head()

,i,five,four,three,two,one,Unnamed: 0,installer_id,Year_month,quotes_count,winning_quotes_count,year_month,closeraterecord_id
0,0,0,0,0,0,0,0,108,201506,27.0,0.0,2015-06-01,0
1,1,0,0,0,0,0,1,108,201601,12.0,0.0,2016-01-01,1
2,2,0,0,0,0,0,2,108,201602,3.0,0.0,2016-02-01,2
3,3,0,0,0,0,0,3,363,201603,20.0,0.0,2016-03-01,3
4,4,0,0,0,0,0,4,363,201604,8.0,0.0,2016-04-01,4


In [19]:
pwd

'C:\\Users\\herbie huang\\Google Drive\\Current_SolarResearch\\working_oct'

In [23]:
import subprocess

## Do some processing in Python

## Set do-file information
dofile = "es_monthly.do"
cmd = ["C:\\Users\\herbie huang\\OneDrive\\Documents\\Stata14\\StataSE-64.exe", "do", dofile]

## Run do-file
subprocess.call(cmd) 

0

In [3]:
import pandas as pd 
es_monthly_ind=pd.read_stata('es_monthly_ind.dta')

In [4]:
es_monthly_ind.describe()

,five,four,three,two,one,avg_sent_score,installer_id,quotes_count,winning_quotes_count,year_month_count,reviews_ct,avg
count,8655.000000,8655.000000,8655.000000,8655.000000,8655.000000,5267.000000,8655.000000,6411.000000,6411.000000,8655.000000,8655.000000,5267.000000
mean,3.191912,0.119237,0.021606,0.015713,0.052340,0.683777,18033.262507,37.991889,0.622992,43.116234,3.400809,4.878830
std,5.474295,0.494591,0.150094,0.151970,0.317666,0.166150,6656.861472,91.817707,1.636259,14.669268,5.832675,0.404377
min,0.000000,0.000000,0.000000,0.000000,0.000000,-0.826800,108.000000,1.000000,0.000000,-1.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.635657,20131.000000,3.000000,0.000000,34.000000,0.000000,5.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.706360,20626.000000,11.000000,0.000000,46.000000,1.000000,5.000000
75%,4.000000,0.000000,0.000000,0.000000,0.000000,0.777200,21357.000000,33.000000,1.000000,55.000000,4.000000,5.000000
max,47.000000,6.000000,2.000000,3.000000,5.000000,0.957600,23027.000000,1417.000000,43.000000,64.000000,52.000000,5.000000


### Note: plenty of NaN 

In [5]:
es_monthly_ind.head(10)

,five,four,three,two,one,avg_sent_score,installer_id,quotes_count,winning_quotes_count,year_month_count,reviews_ct,avg
0,0,0,0,0,0,NaN,108,27.0,0.0,30.0,0.0,NaN
1,0,0,0,0,0,NaN,108,NaN,NaN,31.0,0.0,NaN
2,0,0,0,0,0,NaN,108,NaN,NaN,32.0,0.0,NaN
3,0,0,0,0,0,NaN,108,NaN,NaN,33.0,0.0,NaN
4,0,0,0,0,0,NaN,108,NaN,NaN,34.0,0.0,NaN
5,0,0,0,0,0,NaN,108,NaN,NaN,35.0,0.0,NaN
6,0,0,0,0,0,NaN,108,NaN,NaN,36.0,0.0,NaN
7,0,0,0,0,0,NaN,108,12.0,0.0,37.0,0.0,NaN
8,0,0,0,0,0,NaN,108,3.0,0.0,38.0,0.0,NaN
9,0,0,0,0,0,NaN,363,20.0,0.0,39.0,0.0,NaN


## Output : es_monthly.dta 